In [1]:
import sys
sys.path.insert(0, '../src/')

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import scipy.sparse as sp
from scipy.sparse import load_npz

import torch

import utils
from net import start_experiments

In [2]:
_A_obs = load_npz('../data/datasets/CORA_ML.npz')

In [3]:
val_share = 0.1
test_share = 0.05
seed = 481516234

train_ones, val_ones, val_zeros, test_ones, test_zeros = utils.train_val_test_split_adjacency(_A_obs, val_share, test_share, seed, undirected=True, connected=True, asserts=False)

train_graph = sp.csr_matrix((np.ones(len(train_ones)),(train_ones[:,0], train_ones[:,1])))
assert (train_graph.toarray() == train_graph.toarray().T).all()

In [4]:
models = start_experiments(num_experiments=5,
                           experiment_root='../logs/CORA-ML/Ours_test_loss',
                           train_graph=train_graph,
                           H=9,
                           optimizer=torch.optim.Adam,
                           optimizer_args={'lr': 0.1,
                                           'weight_decay': 1e-7},
                           invoke_every=9,
                           steps = 36,
                           val_edges=(test_ones, test_zeros))


Experiment_0
Step:  9/36, Loss: 6.33558, Edge-Overlap: 0.024
Step: 18/36, Loss: 4.30382, Edge-Overlap: 0.259
Step: 27/36, Loss: 3.47224, Edge-Overlap: 0.410
Step: 36/36, Loss: 3.06890, Edge-Overlap: 0.516

Experiment_1
Step:  9/36, Loss: 6.40830, Edge-Overlap: 0.024
Step: 18/36, Loss: 4.40318, Edge-Overlap: 0.255
Step: 27/36, Loss: 3.56162, Edge-Overlap: 0.400
Step: 36/36, Loss: 3.13698, Edge-Overlap: 0.500

Experiment_2
Step:  9/36, Loss: 6.38015, Edge-Overlap: 0.019
Step: 18/36, Loss: 4.38033, Edge-Overlap: 0.238
Step: 27/36, Loss: 3.48905, Edge-Overlap: 0.404
Step: 36/36, Loss: 3.05927, Edge-Overlap: 0.513

Experiment_3
Step:  9/36, Loss: 6.34400, Edge-Overlap: 0.023
Step: 18/36, Loss: 4.34130, Edge-Overlap: 0.249
Step: 27/36, Loss: 3.54027, Edge-Overlap: 0.404
Step: 36/36, Loss: 3.13242, Edge-Overlap: 0.505

Experiment_4
Step:  9/36, Loss: 6.34570, Edge-Overlap: 0.024
Step: 18/36, Loss: 4.30256, Edge-Overlap: 0.251
Step: 27/36, Loss: 3.46119, Edge-Overlap: 0.418
Step: 36/36, Loss: